<a href="https://colab.research.google.com/github/yutawatabe/pyCGE/blob/main/EK_Fullsolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 均衡の計算

シミュレートした適当なパラメーターから均衡の計算を行います。

これらのステップを踏みます
1. パラメータの設定
2. パラメータと、ある賃金の下での（超過）労働需要の計算
3. 超過労働需要をもとに、賃金のアップデート
4. 2から3を関数にして、ループする。超過労働需要が収束することを確認する
5. 超過労働需要がない状態を確認し、賃金と貿易を出力する

In [2]:
# numpy, mathをインポート
import numpy as np # 行列計算のライブラリ
from math import gamma # 価格指数のため

# パラメーターの設定

In [3]:
N = 3 # 国の数
theta = 4 # 貿易弾力性のパラメーター
sigma = 3 # 代替の弾力性のパラメーター

In [4]:
# np.arrayはベクトルを作るための関数。ここではベクトル（一次元）のものしか扱わないが、多次元の行列ももちろん作ることができる。
T = np.array([1., 1., 1.]) # 技術パラメーター

# printを行うことで、Tの中身を表示させる。
# T.shapeはTのshape（ベクトルの形）を求めるもの。それをprintして、行列が正しい形化を確認する。
print(T)
print(T.shape)

[1. 1. 1.]
(3,)


In [5]:
L_S = np.array([1,1.5,1.5]) # 人口
print(L_S)
print(L_S.shape)

[1.  1.5 1.5]
(3,)


$\boldsymbol{d}$を設定します。まず $\boldsymbol{d}$のすべての要素を1.5として、そのあとにfor ループを使用して国内貿易費用を1に変更します。

In [6]:
d = np.ones((N,N)) * 1.5 # 貿易費用の初期設定を作る。
print(d)
print(d.shape)
# Pythonにおけるfor loopは以下のように書ける。
# 細かく言えば、ここではORという変数を用意して、それが0からN-1まで順に上がっていくループになっている。
# d[OR,OR] = 1 とすることで、 d[0,0] = 1, d[1,1] = 1, ... のように国内貿易費用を1にする動作をそれぞれの国で行っている。
for OR in np.ndindex((N)):
  d[OR,OR] = 1 #国内貿易費用は1

print(d)

[[1.5 1.5 1.5]
 [1.5 1.5 1.5]
 [1.5 1.5 1.5]]
(3, 3)
[[1.  1.5 1.5]
 [1.5 1.  1.5]
 [1.5 1.5 1. ]]


#（超過）労働需要の計算

仮に国の賃金が同一として、果たして労働市場（財市場）が均衡するかを見ます。候補として全ての国 $i$ に対して
$$ w_i = 0.25 $$
を考えます


In [7]:
w = np.ones((N)) / 4
print(w)

[1. 1. 1.]


まず総所得（消費）を計算しておきます。$X_n$を総消費とします。この経済では労働のみが生産要素なので、
$$Y_n = X_n = w_n L_n$$
となります。

In [8]:
# w * L でベクトル同士の要素の掛け算になる（内積を取るわけではないことに注意）。
Xn = w * L_S
print(Xn)

[1.  1.5 1.5]


輸入シェアを計算する前に、二次元のループについて説明します。
ループは多次元の変数について行うことができます。これを輸入シェアの計算に活用します。


In [9]:
# 一次元のループは以下のように書きます。
for OR in np.ndindex((N)):
  print(OR)

# 二次元のループは以下のように書けます。
# ここではOR, DEをループする変数として使っています。
# np,ndindex(N,N)とは、N×N行列の要素をそれぞれを追っていくようなイメージです。
# 具体的には　OR = 0, DE = 0 から OR = 0, DE = 1のように変数が動いていきます。
for OR,DE in np.ndindex((N,N)):
  print([OR,DE])

(0,)
(1,)
(2,)
[0, 0]
[0, 1]
[0, 2]
[1, 0]
[1, 1]
[1, 2]
[2, 0]
[2, 1]
[2, 2]


輸入シェアを計算します。数式は以下のようになります
$$ \pi_{ni} = \frac{ T_{i} (w_{i} d_{ni})^{-\theta} }{\sum_{k=1} T_{k} (w_{k} d_{ki})^{-\theta} } = \frac{X_{ni}}{X_i}  $$
ここで$X_{ni}$はi国からn国への輸出額になります。コーディングの際には以下のような表記を用います。
$$ \pi_{ni} = \frac{\pi_{ni,num}}{\Phi_{n}} $$
ここでは
$$ \pi_{ni,num} = T_{i} (w_{i} d_{ni})^{-\theta} $$
であり、
$$ \Phi_{n} = \sum_{k=1}^N T_{k} (w_{k} d_{nk})^{-\theta} $$
となります。


In [10]:
# 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）の初期の箱を作っておく。
# この箱がないと、ループしたときに埋める箱がないのでエラーが出る。
pi = np.zeros((N,N))
pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
Phi = np.zeros((N)) # 輸入シェアの分母部分

for OR,DE in np.ndindex((N,N)):
  pi_num[OR,DE] = T[OR] * (w[OR] * d[OR,DE]) ** (-theta)
  # このpi_num[OR,DE]を足し上げてPhi[DE]を作っていく。
  # この再帰を繰り返すと、Phi[DE]はpi_num[OR,DE]を最初に次元で足し上げていることになる。
  Phi[DE] = Phi[DE] +  pi_num[OR,DE]

# 分子を分母で分かって輸入シェアを求める。
for OR,DE in np.ndindex((N,N)):
  pi[OR,DE] = pi_num[OR,DE] / Phi[DE]

print(pi)

[[0.71681416 0.14159292 0.14159292]
 [0.14159292 0.71681416 0.14159292]
 [0.14159292 0.14159292 0.71681416]]


ここで、$\pi_{in}$の定義上、$ \sum_{i=1}^N \pi_{in} = 1 $ が成立するはず。
また仮置きした賃金が同一であり、国の技術、貿易費用が対象のため、$\boldsymbol{\pi}$も対称になります。それを上で確認します。

ここから価格指数を計算しておきます（あとで厚生の評価に使う）
$$ P_{n} = \Gamma \left( \frac{\theta + \sigma -1}{\theta} \right) \Phi_{n}^{-1/\theta} $$

In [11]:
P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)
print(P)

[0.97741539 0.97741539 0.97741539]


$i$ 国から $n$ 国への輸出は、輸入シェアに輸入国の総消費をかけたものになります。
$$X_{ni} = \pi_{ni} X_n $$

In [12]:
X = np.zeros((N,N))
for OR,DE in np.ndindex((N,N)):
  X[OR,DE] = pi[OR,DE] * Xn[DE]

print(X)

[[0.71681416 0.21238938 0.21238938]
 [0.14159292 1.07522124 0.21238938]
 [0.14159292 0.21238938 1.07522124]]


ここでは$\boldsymbol{\pi}$のように足して1にならず、対称にもなりません。これは需要規模が国によって違うためです（消費者の数が違う）。

労働需要は、総売上を賃金で割ると導けます。数学的には

$$ L_{i,D} = \frac{ \sum_{n=1}^N X_{ni}}{w_i} $$


In [14]:
L_D = np.zeros((N))
for OR,DE in np.ndindex((N,N)):
  L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

print(L_D)

[1.14159292 1.42920354 1.42920354]


**労働市場の均衡の確認**

まず労働供給は固定であり、$\boldsymbol{L} = \boldsymbol{L}_S$になります。労働の超過需要をZを求めてあげます。

$$ Z_{i} = L_{i,D} - L_i $$

この賃金が労働市場を均衡させる賃金なら、Zはどの国でも0になるはずです。


In [15]:
Z = L_D - L_S
print(Z)

[ 0.14159292 -0.07079646 -0.07079646]


ならない、ということはこの賃金は均衡賃金ではありません！
次の賃金候補を探してあげます。今の賃金を $w^0_i$とし（最初のinterationの賃金）、アップデート方法を考えます。ここでは超過需要を用いたアップデータ方を用います。

$$ w_{new,i} = w_i * \left(1 + \psi * \frac{Z_i}{L_i}\right) $$

In [18]:
psi = 0.1 # 収束のスピードをコントロールするパラメーター

w_new = w * (1 + psi * (Z / L_S) )
print(w_new)

[1.01415929 0.99528024 0.99528024]


ここで新しい$w$と古い$w$を比較します。ちゃんと超過需要が発生していた国において賃金が高くなっているか確認できるはずです。

In [21]:
print(Z)
print(w_new - w)

[ 0.14159292 -0.07079646 -0.07079646]
[-0.74646018 -0.75117994 -0.75117994]


均衡賃金は基準化なしに一意にはきまりません。最後に世界のGDPが1になるように基準化しておきます。

In [22]:
# np.sumは足し算を取る関数。これは一次元のベクトルなので、要素をすべて足し上げている。
wgdp = np.sum(w_new * L_S)
print(wgdp)
# それぞれの国の賃金を現在の計算での全世界のGDPで割ることにより、改めて世界全体のGDPが1になる。
w_new = w_new / wgdp
print (np.sum(w_new * L_S))


1.0
1.0


# 超過労働需要計算の関数

いままでの流れをまとめると

1.   パラメーターを設定する
2.   仮定した賃金をもとに、輸入シェア、貿易額を計算する
3.   貿易額を用いて、労働の超過需要を計算し、仮置きした賃金を更新していく。

この2から3を**関数**にまとめます。


In [24]:
def updatewage(w,theta,sigma,N,L_S,T,d,psi):
    """
    パラメータと経済変数に基づいて賃金を更新します。

    この関数は、入力パラメータと経済変数に基づいて新しい賃金値を計算します。

    パラメータ:
        w (np.ndarray): 異なる国の現在の賃金率の配列。
        theta (float): 貿易の弾力性。
        sigma (float): 財の代替の弾力性。
        N (int): 国の数。
        L_S (np.ndarray): 各国の労働供給。
        T (np.ndarray): 各国の技術水準。
        d (np.ndarray): 国々間の貿易コスト。
        psi (float): 賃金調整パラメータ。

    戻り値:
        w_new (np.ndarray): 更新された国の賃金。
        Z (np.ndarray): 各国の超過労働需要
        P (np.ndarray): 価格指数。
        X (np.ndarray): 国々間の貿易フロー。
    """
    Xn = w * L_S

    pi = np.zeros((N,N)) # 輸入シェア（n国の総消費のうち、i国の財が何割を占めるか）
    pi_num = np.zeros((N,N)) # 輸入シェアの分子部分
    pi_den = np.zeros((N)) # 輸入シェアの分母部分
    for OR,DE in np.ndindex((N,N)):
      pi_num[OR,DE] = T[OR] * (w[OR] * d[OR,DE]) ** (-theta)
      pi_den[DE] += pi_num[OR,DE] # pi_den[DE] = pi_den[DE] + pi_num[OR,DE]

    for OR,DE in np.ndindex((N,N)):
      pi[OR,DE] = pi_num[OR,DE] / pi_den[DE]

    P = gamma((theta + sigma - 1) / theta) **(1/(1-sigma)) * Phi ** (-1/theta)

    X = np.zeros((N,N))
    for OR,DE in np.ndindex((N,N)):
      X[OR,DE] = pi[OR,DE] * Xn[DE]

    L_D = np.zeros((N))
    for OR,DE in np.ndindex((N,N)):
      L_D[OR] += X[OR,DE] / w[OR] # L_D[OR] = L_D[OR] + X[OR,DE] / w[OR]

    Z = L_D - L_S
    w_new = w * (1 + psi * (Z / L_S) )

    wgdp = np.sum(w_new * L_S)
    w_new = w_new / wgdp

    return w_new,Z,P,X

ここでちゃんと結果が前の関数なしで書いたiterationと一致するかを確認します。

In [26]:
w_newfunc,_,_,_ = updatewage(w,theta=theta,sigma=sigma,N=N,L_S=L_S,T=T,d=d,psi=psi)
print(w_newfunc)
print(w_new)

[0.25353982 0.24882006 0.24882006]
[0.25353982 0.24882006 0.24882006]


# While-loopの導入

ここではwhile loopを用いて、賃金が超過需要を十分に小さくするまで、更新を続けていくようなコードを書いていきます。ここで閾値tolを定義します。While loopとは、ある条件を満たすまで(While)、繰り返し計算を行うようなループになります。ここでは
1. 仮置きした賃金を使って超過労働需要を計算
2. 超過労働需要から賃金をアップデート
3. 超過労働需要がtolより大きければ、1に戻る
4. 超過労働需要がtolより小さければ、経済の均衡アウトカムを計算して、出力する

In [29]:
tol = 0.0001
iter = 1

# 超過労働需要関数と賃金の初期値を設定してあげる。
Z = np.ones((N))
w = np.ones((N))

# 超過労働需要がtolより大きければ、ここに戻る
while max(np.abs(Z)) > tol:
  iter += 1
  w_old = np.copy(w)
  w,Z,P,X = updatewage(w,theta=4,sigma=3,N=3,L_S=L_S,T=T,d=d,psi=0.1)
  if iter % 10 == 0:
    print(iter)
    print(Z)
    print(w)



10
[ 0.00559218 -0.00294981 -0.00294981]
[0.26031388 0.24656204 0.24656204]
20
[ 1.13421104e-04 -5.99655974e-05 -5.99655974e-05]
[0.26061275 0.24646242 0.24646242]


最後にもう一度賃金を更新し、超過労働需要がゼロであることを確認し、均衡アウトカムと、厚生を計算します。厚生は
$$ U_n = \frac{w_n}{P_n} $$
となります。

In [31]:
w,Z,P,X = updatewage(w,theta=4,sigma=3,N=3,L_S=L_S,T=T,d=d,psi=0.1)
print(Z)
print(w)

U = w / P
print(U)

[ 5.20614440e-05 -2.75255293e-05 -2.75255293e-05]
[0.26061611 0.2464613  0.2464613 ]
[0.26663802 0.25215615 0.25215615]
